# Fluorescence Spectra Viewer

This notebook shows how to create a basic **fluorescence spectra viewer**. For more complex visualizations, use one of the listed online spectra viewers.

## Introduction

Labeling and visualizing cells and molecules with [fluorescent proteins](http://www.scholarpedia.org/article/Fluorescent_proteins) is one of the most common approaches in modern neuroscience to study the nervous system. Other techniques like [optogenetics](https://spikesandbursts.wordpress.com/2018/07/24/optogenetics-as-a-candle-in-the-brain/) use light-sensitive proteins that respond to specific wavelengths, allowing scientists to manipulate neuronal activity. All these techniques require the proper selection of light sources and microscope filters to match the optimal excitation and emission properties of the fluorescent or light-sensitive proteins. Although there are many nuances, one way to select the correct setup is to plot the **fluorescence spectra** of our protein of interest together with the available light sources and filters in the laboratory.

## Fluorescent Proteins

Fluorescent proteins started to become widely used in neuroscience thanks to the engineering of the green fluorescent protein (GFP) in the 1990s by several groups, including Roger Tsien, Osamu Shinomura, and Martin Chalfie, who were awarded the Nobel Prize in 2008 for the discovery and development of GFP. Nowadays, there are hundreds of available fluorescent proteins (see [FPbase](https://www.fpbase.org/)) used for labeling or as part of other biosensors such as [calcium indicators](https://www.nature.com/articles/s41586-023-05828-9), [optogenetics](https://spikesandbursts.wordpress.com/2018/07/24/optogenetics-as-a-candle-in-the-brain/), [voltage indicators](https://spikesandbursts.wordpress.com/2018/11/06/voltage-imaging-to-record-them-all/), [DREADDs](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4759656/), and [GRAB sensors](https://www.annualreviews.org/content/journals/10.1146/annurev-neuro-110520-031137), among others.

* [FPbase](https://www.fpbase.org/) is one of the best open-source **fluorescent protein databases**. You can download the spectra of the proteins, explore their properties and relationships with other proteins, read the references, and access many other useful tools.

## Equipment

For this notebook, the two main pieces of equipment we need to consider are the light source and the filters. They are key components in the most common configuration in fluorescence microscopy: light source > excitation filter > dichroic filter (between excitation and emission filter) > emission filter > camera or detector.

The **light sources** are generally lasers (1-2 nm bandwidth, more powerful, more expensive) and LEDs (wider spectral widths, less powerful, less expensive).

The **optical filters** selectively transmit light at particular wavelengths. Depending on this, they can be longpass (transmit all wavelengths longer than a specific wavelength), shortpass (transmit all wavelengths shorter than a specific wavelength), or bandpass/multiband (transmit across one or more specific ranges of wavelengths). Other important properties for selecting a filter are transmission percentage (ideally >85%) and optical density (the background darkness, preferably > 3.0).

* [Thorlabs](https://www.thorlabs.com/) has a lot of technical information, specifications, and performance data on both [LEDs](https://www.thorlabs.com/newgrouppage9.cfm?objectgroup_id=2692), [lasers](https://www.thorlabs.com/navigation.cfm?guide_id=31), and [optical filters](https://www.thorlabs.com/navigation.cfm?guide_id=21).

## More Resources

* Online spectra viewers: [ThermoFisher](https://www.thermofisher.com/order/fluorescence-spectraviewer/), [FPbase](https://www.fpbase.org/spectra/), [Chroma](https://www.chroma.com/spectra-viewer).
* More resources (tutorials, courses, etc.): [Spikes and Bursts blog](https://spikesandbursts.wordpress.com/resources/#microscopy).

# Example data

Fluorescent protein and LED spectra, and filter bandpass data.

Here are the sources of the example_data:

* [mapple spectra (FPbase)](https://www.fpbase.org/spectra_csv/?q=69,70). Renamed to "fp_spectra_mapple.csv".
* [EGFP spectra (FPbase)](https://www.fpbase.org/spectra_csv/?q=17,18,173). Renamed to "fp_spectra_egfp.csv".
* [LED 470 nm (Thorlabs)](https://www.thorlabs.com/images/TabImages/M470L5_Data.xlsx). Renamed to "led_spectrum_M470L5.xlsx".
* [LED 565 nm (Thorlabs)](). Renamed to "led_spectrum_M565L3.xlsx"
* [Filter FF01-479/585-25 (Semrock)](https://www.idex-hs.com/store/product-detail/ff01_479_585_25/fl-004029). Renamed to "filter_spectrum_479-585.txt"
* [Filter FF01-479/585-25 (Semrock)](https://www.idex-hs.com/store/product-detail/ff01_524_628_25/fl-004030). Renamed to "filter_spectrum_524_628.txt"/

# Import the libraries

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# Interactive plots
import ipywidgets as widgets
from IPython.display import display
#For Jupyter Lab:
%matplotlib widget

import os
import sys

# Create the paths

In [ ]:
notebook_name = 'spectra_viewer'

# Data path to 'Data_example' folders. Change accordingly to your data structure.
data_path = os.path.dirname(os.getcwd())  # Moves one level up from the current directory

# Change the folder names accordingly
paths = {'data': data_path,
         'raw_data':  f'{data_path}/Data_examples/{notebook_name}/',
         'processed_data': f'{data_path}/Processed_data_examples/{notebook_name}/',
         'analysis': f'{data_path}/Analysis_examples/{notebook_name}/',         
         'plots': f'{data_path}/Analysis_examples/{notebook_name}/Plots/'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Color maps and wavelength range

In [ ]:
# Define the wavelength range for the plot and color map
wavelength_start = 350
wavelength_end = 750

# Define the new colormap with the desired range
color_map_range = (wavelength_start, wavelength_end)
custom_colormap = colors.LinearSegmentedColormap.from_list('spectral_custom',
                                                           colors=['violet', 'indigo', 
                                                                   'blue', 'green', 'yellow', 
                                                                   'orange', 'red', 'darkred'],
                                                           N=128)  # number of colors

# Load fluorescent protein spectra


In [ ]:
# Select the fluorescent protein spectra to plot
fp_spectra = ['fp_spectra_mapple', 'fp_spectra_egfp']  # or blank
fp_file_extension = 'csv'  # Change the pandas function to load the data accordingly

# Define here the column indexes
fp_wavelength_column = 0
fp_ex_intensity_column = 1
fp_em_intensity_column = 2

In [ ]:
# Empty list to store fluorescent protein data
fp_data = []

for fp_spectrum in fp_spectra:
    file_path = os.path.join(paths['raw_data'], f"{fp_spectrum}.{fp_file_extension}")  # Another way to create the path
    print(file_path)

    # Load the data
    fp_spectra_data = pd.read_csv(file_path, delimiter=",", keep_default_na=True)
    
    # Fluorescent protein excitation and emission spectra
    fp_ex = fp_spectra_data.iloc[:, [fp_wavelength_column, fp_ex_intensity_column]] 
    fp_ex_label = fp_spectra_data.columns[fp_ex_intensity_column]
    
    fp_em = fp_spectra_data.iloc[:, [fp_wavelength_column, fp_em_intensity_column]] 
    fp_em_label = fp_spectra_data.columns[fp_em_intensity_column]

    # Find the wavelength with peak excitation and emission intensities
    fp_ex_peak = fp_ex.loc[fp_ex.iloc[:, 1].idxmax()]
    fp_em_peak = fp_em.loc[fp_em.iloc[:, 1].idxmax()]

    # Assign colors based on the peak wavelengths
    fp_ex_peak_proportion = ((fp_ex_peak[fp_ex.columns[fp_wavelength_column]] - wavelength_start) 
                             / (wavelength_end - wavelength_start))
    fp_ex_color = custom_colormap(fp_ex_peak_proportion)
    
    fp_em_peak_proportion = ((fp_em_peak[fp_em.columns[fp_wavelength_column]] - wavelength_start) 
                             / (wavelength_end - wavelength_start))
    fp_em_color = custom_colormap(fp_em_peak_proportion)

    fp_data.append((fp_ex, fp_ex_label, fp_ex_color, fp_em, fp_em_label, fp_em_color))


# Load filters data

In [ ]:
filternames = ['filter_spectrum_479-585', 'filter_spectrum_524_628']  # or blank
filter_file_extension = 'txt'  # Change pandas the function to load the data accordingly

filter_wavelength_column = 0
filter_intensity_column = 1
filter_data_row = 4

In [ ]:
# Empty list to store filtered data
filter_data = []

for filtername in filternames:
    file_path = f"{paths['raw_data']}{filtername}.{filter_file_extension}"
    print(file_path)
    
    # Load the data
    filter_spectrum_file = pd.read_csv(file_path, delimiter="\t", header=None, keep_default_na=True)
    
    # Select rows with data and convert columns to float
    filter_spectrum = filter_spectrum_file.iloc[filter_data_row:, [filter_wavelength_column, filter_intensity_column]]
    filter_spectrum.columns = ['wavelength', 'transmission']
    filter_spectrum = filter_spectrum.astype(float)
    
    # Append the filtered data along with its label
    filter_data.append((filter_spectrum, filtername))

## Load the LED spectrum data

In [ ]:
led_spectra = ['led_spectrum_M470L5', 'led_spectrum_M565L3']
led_file_extension = 'xlsx'  # Change the pandas function to load the data accordingly

# Change the column indices accordingly to the tabular data
led_column_header = 1
led_wavelength_column = 2
led_intensity_column = 3

In [ ]:
# Empty list to store led data
led_data = []

for led_spectrum in led_spectra:
    file_path = f"{paths['raw_data']}{led_spectrum}.{led_file_extension}"
    print(file_path)

    # Load the data
    led_spectra_data = pd.read_excel(file_path, keep_default_na=True, header=led_column_header)
    
    # LED spectra data (change column index accordingly)
    led_spectrum_data = led_spectra_data.iloc[:, [led_wavelength_column, led_intensity_column]]

    # Find the LED peak
    led_peak = led_spectrum_data.loc[led_spectrum_data.iloc[:, 1].idxmax()]

    # Calculate the peak's proportion within the wavelength range to assign the corresponding color
    led_peak_proportion = ((led_peak[led_spectrum_data.columns[0]] - wavelength_start) 
                           / (wavelength_end - wavelength_start))
    
    # led_color = plt.get_cmap('Spectral_r')(led_peak_proportion)
    led_color = custom_colormap(led_peak_proportion)
    
    led_data.append((led_spectrum_data, led_spectrum, led_color))

# Spectra viewer

You can modify the viewer by changing the colormap, separating spectra in different subplots (ax1, ax2, etc.), changing the wavelength range, etc.

In [ ]:
# Plot parameters
fig, ax = plt.subplots(figsize=(10, 4), sharex=True)
plt.rcParams['font.size'] = 12
ax.spines['top'].set_visible(False)    # Remove top frame
ax.spines['right'].set_visible(False)  # Remove right frame
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Normalized values')
ax.set_xlim(wavelength_start, wavelength_end)
ax.set_ylim(0, 1)

# Fluorescent protein spectra (comment out if needed)
for fp_ex, fp_ex_label, fp_ex_color, fp_em, fp_em_label, fp_em_color in fp_data:
    # Plot the spectra
    ax.fill_between(fp_ex.iloc[:, 0], fp_ex.iloc[:, 1], 
                    label=fp_ex_label, color=fp_ex_color, alpha=0.3, linewidth=0)
    ax.fill_between(fp_em.iloc[:, 0], fp_em.iloc[:, 1], 
                    label=fp_em_label, color=fp_em_color, alpha=0.3, linewidth=0)

# Filtersets with varying linestyles
linestyles = ['dashed', 'solid', 'dashdot']  # Define linestyles
for index, (filter_spectrum, filtername) in enumerate(filter_data): 
    linestyle = linestyles[index % len(linestyles)]  
    ax.plot(filter_spectrum['wavelength'], filter_spectrum['transmission'], 
            label=filtername, color='black', linestyle=linestyle, linewidth=1)

# LED spectra (comment out if needed)
for led_spectrum_data, led_spectrum, led_color in led_data:
    # Plot the LED spectrum
    ax.plot(led_spectrum_data.iloc[:, 0], led_spectrum_data.iloc[:, 1], 
            label=led_spectrum, color=led_color, linewidth=2)

# Add colorbar below x-axis
cax = plt.axes([0.125, -0.10, 0.775, 0.05])
cbar = plt.colorbar(plt.cm.ScalarMappable(norm=None, cmap=custom_colormap),
                     cax=cax, orientation='horizontal')
cbar.set_ticks([])
cbar.set_ticklabels([])

# Add legend and display plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=False)
fig.tight_layout()
plt.show()

## Save the plot

In [ ]:
fig.savefig(f"{paths['plots']}/Spectra_plot.tif", dpi=300)

# Custom LED and filters

An example of how to create simple Gaussian spectra and rectangular filter bands.

In [ ]:
wavelength_range = np.linspace(wavelength_start, wavelength_end, wavelength_end - wavelength_start)

# Calculate Gaussian spectrum for LEDs
led_peak_wavelength = 470
led_fwhm = 40
led_sigma = led_fwhm / (2 * np.sqrt(2 * np.log(2)))
led_spectrum = np.exp(-(wavelength_range - led_peak_wavelength)**2 / (2 * led_sigma**2))

# Define the bandpass filter spectrum for the excitation filter
ex_filter_center = 468
ex_filter_width = 40

ex_filter_band = np.zeros_like(wavelength_range)
ex_filter_band[(wavelength_range >= ex_filter_center - ex_filter_width/2) 
                           & (wavelength_range <= ex_filter_center + ex_filter_width/2)] = 1

# Define the cutoff wavelength for the longpass filter
longpass_cutoff = 500  # Example cutoff wavelength

# Create the longpass filter spectrum
longpass_filter = np.zeros_like(wavelength_range)
longpass_filter[wavelength_range >= longpass_cutoff] = 1

# Plot the custom LED and filter
fig, ax = plt.subplots(figsize=(8,4))

ax.plot(wavelength_range, led_spectrum)

ax.plot(wavelength_range, ex_filter_band, 
        linestyle='dashed', color='black',)
ax.plot(wavelength_range, longpass_filter, 
        linestyle='dotted', color='black')

ax.set_ylabel('Normalized values')
ax.set_xlabel('Wavelenth (nm)')

fig.tight_layout()
plt.show()